In [1]:
import os
import sys
import plotly

Initialize spark session.

In [2]:
spark_home = os.environ.get('SPARK_HOME',None)
sys.path.insert(0,spark_home + "/python")
sys.path.insert(0,os.path.join(spark_home,'python/lib/py4j-0.8.2.1-src.zip'))
with open(os.path.join(spark_home,'python/pyspark/shell.py')) as source_file:
    exec(source_file.read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.2.0
      /_/

Using Python version 3.6.1 (default, May 11 2017 13:25:24)
SparkSession available as 'spark'.


Load data from csv to Spark dataframe.

In [3]:
rawData = sqlContext.read.load(r'C:\Users\Nick\Documents\Projects\LTBP\LDM & Deterioration Modeling\LTBP Bridge Portal export.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

In [9]:
type(sqlContext)

pyspark.sql.context.SQLContext

In [38]:
dir(rawData)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_jcols',
 '_jdf',
 '_jmap',
 '_jseq',
 '_lazy_rdd',
 '_sc',
 '_schema',
 '_sort_cols',
 'agg',
 'alias',
 'approxQuantile',
 'cache',
 'checkpoint',
 'coalesce',
 'collect',
 'columns',
 'corr',
 'count',
 'cov',
 'createGlobalTempView',
 'createOrReplaceGlobalTempView',
 'createOrReplaceTempView',
 'createTempView',
 'crossJoin',
 'crosstab',
 'cube',
 'describe',
 'distinct',
 'drop',
 'dropDuplicates',
 'drop_duplicates',
 'dropna',
 'dtypes',
 'explain',
 'fillna',
 'filter',
 'first',
 'foreach',
 'foreachPartition',
 'freqItems',
 'groupBy',
 'groupby',
 'head',
 'hint',
 'intersect'

Raw data schema.

In [8]:
rawData.printSchema()

root
 |-- 1 - State Name: string (nullable = true)
 |-- 8 - Structure Number: string (nullable = true)
 |-- 2 - Highway Agency District: string (nullable = true)
 |-- 5A - Record Type: string (nullable = true)
 |-- 5B - Route Signing Prefix: string (nullable = true)
 |-- 5C - Designated Level of Service: string (nullable = true)
 |-- 5D - Route Number: string (nullable = true)
 |-- 5E - Directional Suffix: string (nullable = true)
 |-- 6A - Features Intersected: string (nullable = true)
 |-- 6B - Critical Facility Indicator: string (nullable = true)
 |-- 7 - Facility Carried By Structure: string (nullable = true)
 |-- 9 - Location: string (nullable = true)
 |-- 10 - Inventory Rte, Min Vert Clearance: double (nullable = true)
 |-- 11 - Kilometerpoint: double (nullable = true)
 |-- 12 - Base Highway Network: string (nullable = true)
 |-- 13A - LRS Inventory Route: string (nullable = true)
 |-- 13B - Subroute Number: integer (nullable = true)
 |-- 19 - Bypass/Detour Length: integer (nulla

Total number of samples in the dataset.

In [11]:
rawData.count()

1202

Determine states in sample dataset.

In [29]:
rawData.groupBy("1 - State Name").count().show(n=50)

+-------------------+-----+
|     1 - State Name|count|
+-------------------+-----+
|       30 - Montana|   39|
|        32 - Nevada|   26|
|      17 - Illinois|   60|
|     55 - Wisconsin|   41|
| 54 - West Virginia|    8|
|  42 - Pennsylvania|   24|
|     47 - Tennessee|   62|
|      21 - Kentucky|   54|
|         48 - Texas|   60|
|          49 - Utah|   51|
|        4 - Arizona|   26|
|     6 - California|   60|
|  44 - Rhode Island|    2|
|      26 - Michigan|   34|
|     27 - Minnesota|   26|
|  46 - South Dakota|   13|
|   28 - Mississippi|   30|
|37 - North Carolina|   50|
|         16 - Idaho|   20|
|        20 - Kansas|    2|
|     22 - Louisiana|   15|
|        1 - Alabama|   24|
|      24 - Maryland|   20|
| 25 - Massachusetts|   15|
|       18 - Indiana|   60|
|       12 - Florida|   28|
|          39 - Ohio|   51|
|       56 - Wyoming|   60|
|        41 - Oregon|   11|
|       5 - Arkansas|   42|
|      51 - Virginia|   33|
|       13 - Georgia|    1|
|      31 - Nebraska

In [73]:
x = rawData.toPandas()["1 - State Name"]
z = rawData.toPandas()["58 - Deck"]
rtg = [int(z[i][0]) for i in range(0,len(z))]
# states = [x[i][4:] for i in range(0,len(x))]

data = [dict(type='choropleth',autocolorscale = True,
        locations = states, z = rtg, locationmode = 'USA-states',
        marker = dict(line = dict(color = 'rgb(255,255,255)',width = 2)),
        colorbar = dict(title = "Deck Condition Ratings"))]

layout = dict(
        title = 'Deck Condition Ratings for Sample Dataset',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
plotly.plotly.iplot(fig, filename='cloropleth-map' )

In [28]:
rawData.groupBy("58 - Deck").count().show(n=50)

+--------------------+-----+
|           58 - Deck|count|
+--------------------+-----+
|  4 - POOR CONDITION|   14|
|6 - SATISFACTORY ...|  285|
|  5 - FAIR CONDITION|   93|
|8 - VERY GOOD CON...|  186|
|3 - SERIOUS CONDI...|    3|
|  7 - GOOD CONDITION|  607|
|9 - EXCELLENT CON...|   14|
+--------------------+-----+



In [46]:
x=rawData.toPandas()['58 - Deck']
data = [plotly.graph_objs.Histogram(x=x)]
plotly.plotly.iplot(data)